This script shows how data from the different model simulation runs and countries is aggregated to country and region level. The example is only done for Latin America and a small model selection of 6 GHM and climate forcing combinations. Real data contains all the countries and 46 model combinateions.
In order to see how damages are generated with CLIMADA to create the input file please the the CLIMADA RiverFlood tutorial here:

https://github.com/CLIMADA-project/climada_python/blob/main/doc/tutorial/climada_hazard_RiverFlood.ipynb

The pre-processing that is additionally necessary to convert asset damages and observed damages is described in the readme.txt under chapter 4 Instructions for use.

In [3]:
import numpy as np
import pandas as pd
import os
# load data (adjust path)
ass_data=pd.read_csv('~/projects/NC_Submission/Climada_papers/climada_papers/202010_flood_attribution/Demo/Demo_Data/DEMO_AssembledDataSubregions.csv')
ass_data[ass_data['D_CliExp_Pos']>0]

,Unnamed: 0,Year,Country,Region,GHM,clim_forc,D_CliExp_Pos,D_CliExp_Neg,D_1980_Pos,D_1980_Neg,D_2010_Pos,D_2010_Neg,D_obs_dummy_Pos,D_obs_dummy_Neg
18,369,1971,ARG,LAM,orchidee,wfdei,1.096388e+06,6.456917e+07,1.669235e+06,1.029778e+08,9.511688e+06,5.980761e+08,NaN,NaN
21,398,1971,ARG,LAM,lpjml,wfdei,8.936264e+05,1.306591e+07,1.359166e+06,2.009086e+07,7.811199e+06,1.204149e+08,NaN,NaN
23,409,1971,ARG,LAM,matsiro,wfdei,3.021842e+05,8.957474e+07,4.560491e+05,1.406277e+08,2.550091e+06,7.986013e+08,NaN,NaN
31,1019,1971,BHS,LAM,lpjml,princeton,3.452119e-02,0.000000e+00,6.223493e-02,0.000000e+00,1.729590e-01,0.000000e+00,NaN,NaN
33,1042,1971,BHS,LAM,lpjml,wfdei,1.872020e-03,0.000000e+00,3.374885e-03,0.000000e+00,9.379248e-03,0.000000e+00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11263,396569,2010,VEN,LAM,lpjml,princeton,2.455575e+09,1.848530e+09,1.133333e+09,1.013319e+09,2.455575e+09,1.848530e+09,1.227788e+08,9.242652e+07
11264,396582,2010,VEN,LAM,orchidee,princeton,2.991682e+09,1.354734e+09,1.436558e+09,7.402328e+08,2.991682e+09,1.354734e+09,1.495841e+08,6.773672e+07
11265,396586,2010,VEN,LAM,lpjml,wfdei,1.990408e+09,8.089158e+07,8.697089e+08,4.454031e+07,1.990408e+09,8.089158e+07,9.952041e+07,4.044579e+06
11266,396591,2010,VEN,LAM,matsiro,princeton,1.003280e+09,1.635081e+09,4.573970e+08,8.996332e+08,1.003280e+09,1.635081e+09,5.016400e+07,8.175404e+07


The data fram contains various time series for different GHMs and Climate forcings (here reduced to 6 combinations from originally 46) for each country of Latin America:

D_CliExp... - Time series accounting for changes in climate and exposure

D_1980... - Time series only accounting for changes in climate (fixed 1980 socio-economic conditions)

D_2010... - Time series only accounting for changes in climate (fixed 1980 socio-economic conditions)

D_obs... - Dummy time series used to simulate observed damages that cannot be provided

Please note, modeled time series start in 1971, but observations only in 1980, all end in 2010.
In the next step data is aggregated to the regional level:


In [25]:
def aggregation_regions(x):
    """
    This function aggregates country-level damages and variables to
    regional level.
    Parameters
    ----------
    x : DataFrame
        country-level damages and other indicators for all model combinations

    Returns
    -------
    DataFrame
        regionally aggregated damages and other indicators
    """
    aggregated_model_damages_pos = x['D_CliExp_Pos'].sum()
    aggregated_model_damages_neg = x['D_CliExp_Neg'].sum()
    aggregated_model_damages_1980_pos = x['D_1980_Pos'].sum()
    aggregated_model_damages_1980_neg = x['D_1980_Neg'].sum()
    aggregated_model_damages_2010_pos = x['D_2010_Pos'].sum()
    aggregated_model_damages_2010_neg = x['D_2010_Neg'].sum()
    aggregated_observed_damages_pos = (x['D_obs_dummy_Pos']).sum()
    aggregated_observed_damages_neg = (x['D_obs_dummy_Neg']).sum()
    return pd.Series([aggregated_model_damages_pos,
                      aggregated_model_damages_neg,
                      aggregated_model_damages_1980_pos,
                      aggregated_model_damages_1980_neg,
                      aggregated_model_damages_2010_pos,
                      aggregated_model_damages_2010_neg,
                      aggregated_observed_damages_pos,
                      aggregated_observed_damages_neg],
                     index=['D_CliExp_Pos', 'D_CliExp_Neg',
                            'D_1980_Pos', 'D_1980_Neg',
                            'D_2010_Pos', 'D_2010_Neg',
                            'D_obs_dummy_Pos',
                            'D_obs_dummy_Neg'])

In [26]:
def region_aggregation(cols, dataFrame):
    """
    This function is a wrapper for the aggregation and selects the columns to
    be aggregated to regional level.

    Parameters
    ----------
    cols : string list
        Columns to be aggregated

    Returns
    -------
    DataFrame
         regionally aggregated damages and other indicators regions
    """
    data_region = dataFrame.groupby(['Year', 'GHM', 'clim_forc', 'Region'])\
                                     [cols].apply(aggregation_regions)\
                                     .reset_index()  # groupby year and model

    return data_region

In [27]:
# define columns to aggregate
agg_cols = ['D_CliExp_Pos', 'D_CliExp_Neg',
                            'D_1980_Pos', 'D_1980_Neg',
                            'D_2010_Pos', 'D_2010_Neg',
                            'D_obs_dummy_Pos',
                            'D_obs_dummy_Neg']
# execute aggregation
reg_data = region_aggregation(agg_cols, ass_data)
reg_data

,Year,GHM,clim_forc,Region,D_CliExp_Pos,D_CliExp_Neg,D_1980_Pos,D_1980_Neg,D_2010_Pos,D_2010_Neg,D_obs_dummy_Pos,D_obs_dummy_Neg
0,1971,lpjml,princeton,LAM,2.945511e+08,3.689331e+08,4.544100e+08,7.535720e+08,9.788589e+08,2.261250e+09,0.000000e+00,0.000000e+00
1,1971,lpjml,wfdei,LAM,5.057604e+08,1.615544e+09,8.113460e+08,3.189673e+09,1.590751e+09,8.641227e+09,0.000000e+00,0.000000e+00
2,1971,matsiro,princeton,LAM,4.009538e+08,3.686333e+08,6.263636e+08,7.416500e+08,1.339336e+09,1.958968e+09,0.000000e+00,0.000000e+00
3,1971,matsiro,wfdei,LAM,4.037848e+08,6.881197e+08,6.456926e+08,1.341611e+09,1.163209e+09,5.457415e+09,0.000000e+00,0.000000e+00
4,1971,orchidee,princeton,LAM,2.973702e+08,3.501032e+08,4.571187e+08,7.174854e+08,9.864467e+08,2.046198e+09,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
235,2010,lpjml,wfdei,LAM,2.271367e+10,1.085733e+10,9.249558e+09,4.208104e+09,2.271367e+10,1.085733e+10,1.135683e+09,5.428664e+08
236,2010,matsiro,princeton,LAM,2.194935e+10,3.691890e+10,8.399946e+09,9.748158e+09,2.194935e+10,3.691890e+10,1.097468e+09,1.845945e+09
237,2010,matsiro,wfdei,LAM,2.528861e+10,1.713264e+10,9.990795e+09,6.840531e+09,2.528861e+10,1.713264e+10,1.264430e+09,8.566319e+08
238,2010,orchidee,princeton,LAM,1.746115e+10,3.591611e+10,7.277341e+09,9.231161e+09,1.746115e+10,3.591611e+10,8.730574e+08,1.795805e+09


The data is now aggregated over all countries, but there still remain time series for each model.
In the next step, we build the model median and quantiles:

In [28]:
def func_median(x):
    """
    This function aggregates the damages and other indicators from the
    different model runs to the model median and adds basic statistics such as
    the one-third and two-third quantiles.

    Parameters
    ----------
    x : DataFrame
        regionally aggregated damages and other indicators for all model
        combinations

    Returns
    -------
    DataFrame
         model medians of regionally aggregated damages and other indicators
    """
    # identify the median of the model data:
    median_model_damages_pos = x['D_CliExp_Pos'].median()  # =quantile(0.5)
    median_model_damages_neg = x['D_CliExp_Neg'].median()
    median_model_damages_1980_pos = x['D_1980_Pos'].median()  # =quantile(0.5)
    median_model_damages_1980_neg = x['D_1980_Neg'].median()
    median_model_damages_2010_pos = x['D_2010_Pos'].median()  # =quantile(0.5)
    median_model_damages_2010_neg = x['D_2010_Neg'].median()
    median_observed_damages_pos = (x['D_obs_dummy_Pos']).mean()  # all the same value
    median_observed_damages_neg = (x['D_obs_dummy_Neg']).mean()  # all the same value
    one_third_quantile_model_damages_pos = x['D_CliExp_Pos'].quantile(0.3)  # 30
    two_third_quantile_model_damages_pos = x['D_CliExp_Pos'].quantile(0.7)
    one_third_quantile_model_damages_neg = x['D_CliExp_Neg'].quantile(0.3)  # 30
    two_third_quantile_model_damages_neg = x['D_CliExp_Neg'].quantile(0.7)
    one_third_quantile_model_damages_1980_pos = x['D_1980_Pos'].quantile(0.3)  # 30
    two_third_quantile_model_damages_1980_pos = x['D_1980_Pos'].quantile(0.7)  # 70
    one_third_quantile_model_damages_1980_neg = x['D_1980_Neg'].quantile(0.3)  # 30
    two_third_quantile_model_damages_1980_neg = x['D_1980_Neg'].quantile(0.7)  # 70

    one_third_quantile_model_damages_2010_pos = x['D_2010_Pos'].quantile(0.3)  # 30
    two_third_quantile_model_damages_2010_pos = x['D_2010_Pos'].quantile(0.7)  # 70
    one_third_quantile_model_damages_2010_neg = x['D_2010_Neg'].quantile(0.3)  # 30
    two_third_quantile_model_damages_2010_neg = x['D_2010_Neg'].quantile(0.7)  # 70

    # flood_vol = x['FloodVol_Flopros'].median()
    return pd.Series([median_model_damages_pos,
                      median_model_damages_neg,
                      median_model_damages_1980_pos,
                      median_model_damages_1980_neg,
                      median_model_damages_2010_pos,
                      median_model_damages_2010_neg,
                      median_observed_damages_pos,
                      median_observed_damages_neg,
                      one_third_quantile_model_damages_pos,
                      two_third_quantile_model_damages_pos,
                      one_third_quantile_model_damages_neg,
                      two_third_quantile_model_damages_neg,
                      one_third_quantile_model_damages_1980_pos,
                      two_third_quantile_model_damages_1980_pos,
                      one_third_quantile_model_damages_1980_neg,
                      two_third_quantile_model_damages_1980_neg,
                      one_third_quantile_model_damages_2010_pos,
                      two_third_quantile_model_damages_2010_pos,
                      one_third_quantile_model_damages_2010_neg,
                      two_third_quantile_model_damages_2010_neg],
                     index=['D_CliExp_Pos',
                            'D_CliExp_Neg',
                            'D_1980_Pos',
                            'D_1980_Neg',
                            'D_2010_Pos',
                            'D_2010_Neg',
                            'D_obs_dummy_Pos',
                            'D_obs_dummy_Neg',
                            'D_CliExp_Pos_onethird_quantile',
                            'D_CliExp_Pos_twothird_quantile',
                            'D_CliExp_Neg_onethird_quantile',
                            'D_CliExp_Neg_twothird_quantile',
                            'D_1980_Pos_onethird_quantile',
                            'D_1980_Pos_twothird_quantile',
                            'D_1980_Neg_onethird_quantile',
                            'D_1980_Neg_twothird_quantile',
                            'D_2010_Pos_onethird_quantile',
                            'D_2010_Pos_twothird_quantile',
                            'D_2010_Neg_onethird_quantile',
                            'D_2010_Neg_twothird_quantile'])

In [29]:
def model_aggregation(cols, dataFrame, years, select_model):
    """
    This function is a wrapper for the multi-model aggregation and provides
    the model median for each region of all variables.

    Parameters
    ----------
    cols : string list
        Columns to be aggregated

    Returns
    -------
    DataFrame
         regionally aggregated model medians
    """

    if select_model:

        dataFrame = dataFrame[dataFrame['GHM'] == select_model]

    data_models = dataFrame[(dataFrame['Year'] <= np.max(years)) &
                            (dataFrame['Year'] >= np.min(years))]
    # Get the median for model and datasets
    data_models = data_models.groupby(['Year', 'Region'])\
                              [cols].apply(func_median).reset_index()

    return data_models

In [30]:
years = np.arange(1971,2011)
med_mod_data = model_aggregation(agg_cols, reg_data, years, None)
med_mod_data

,Year,Region,D_CliExp_Pos,D_CliExp_Neg,D_1980_Pos,D_1980_Neg,D_2010_Pos,D_2010_Neg,D_obs_dummy_Pos,D_obs_dummy_Neg,...,D_CliExp_Neg_onethird_quantile,D_CliExp_Neg_twothird_quantile,D_1980_Pos_onethird_quantile,D_1980_Pos_twothird_quantile,D_1980_Neg_onethird_quantile,D_1980_Neg_twothird_quantile,D_2010_Pos_onethird_quantile,D_2010_Pos_twothird_quantile,D_2010_Neg_onethird_quantile,D_2010_Neg_twothird_quantile
0,1971,LAM,4.023693e+08,5.285264e+08,6.360281e+08,1.047591e+09,1.251272e+09,3.859332e+09,0.000000e+00,0.000000e+00,...,3.687832e+08,1.151832e+09,5.417412e+08,7.285193e+08,7.476110e+08,2.265642e+09,1.074828e+09,1.465043e+09,2.153724e+09,7.049321e+09
1,1972,LAM,6.467142e+08,7.232123e+08,1.028985e+09,1.028685e+09,2.866084e+09,3.587172e+09,0.000000e+00,0.000000e+00,...,3.595028e+08,1.084149e+09,8.378755e+08,1.311003e+09,6.310323e+08,1.623574e+09,2.099250e+09,3.692527e+09,2.577915e+09,5.066493e+09
2,1973,LAM,8.151027e+08,1.165546e+09,1.210916e+09,1.800896e+09,3.123314e+09,6.070501e+09,0.000000e+00,0.000000e+00,...,6.909524e+08,1.911286e+09,1.111558e+09,1.556728e+09,1.080324e+09,3.017047e+09,2.812444e+09,3.762174e+09,3.798410e+09,9.475175e+09
3,1974,LAM,2.469297e+08,5.895776e+09,3.416155e+08,8.737634e+09,9.556804e+08,3.003573e+10,0.000000e+00,0.000000e+00,...,4.884319e+09,6.622060e+09,3.127346e+08,6.116145e+08,7.246017e+09,9.776114e+09,7.642482e+08,1.325569e+09,2.436126e+10,3.496478e+10
4,1975,LAM,7.356679e+08,1.252483e+09,9.627596e+08,1.854030e+09,2.283276e+09,5.245553e+09,0.000000e+00,0.000000e+00,...,6.635218e+08,1.829390e+09,6.939798e+08,1.423361e+09,9.299430e+08,2.732020e+09,1.460793e+09,3.653704e+09,3.402896e+09,7.534373e+09
5,1976,LAM,5.690548e+08,1.089701e+09,7.273545e+08,1.525657e+09,1.588228e+09,4.114654e+09,0.000000e+00,0.000000e+00,...,8.693011e+08,1.837960e+09,4.706032e+08,1.768818e+09,1.217419e+09,2.594562e+09,1.044985e+09,3.673838e+09,3.100373e+09,6.283568e+09
6,1977,LAM,5.887558e+08,7.184427e+08,6.765118e+08,8.721162e+08,1.528850e+09,3.321514e+09,0.000000e+00,0.000000e+00,...,6.385416e+08,7.927636e+08,6.214238e+08,7.367437e+08,7.746534e+08,9.972082e+08,1.479644e+09,1.741990e+09,2.832951e+09,3.667434e+09
7,1978,LAM,5.674043e+08,1.005163e+09,6.727797e+08,1.187181e+09,1.539537e+09,3.648889e+09,0.000000e+00,0.000000e+00,...,8.895946e+08,1.542610e+09,4.567761e+08,1.356205e+09,1.071408e+09,1.830328e+09,9.577014e+08,3.106723e+09,2.920028e+09,6.619550e+09
8,1979,LAM,9.428826e+08,6.272667e+09,1.009824e+09,6.594910e+09,2.677416e+09,2.195303e+10,0.000000e+00,0.000000e+00,...,6.120509e+09,6.469786e+09,7.945767e+08,1.244181e+09,6.397543e+09,6.764037e+09,1.861484e+09,3.518487e+09,2.132657e+10,2.279633e+10
9,1980,LAM,6.909657e+08,6.501249e+09,6.909657e+08,6.501249e+09,1.854897e+09,2.347668e+10,1.078941e+09,9.669952e+09,...,5.976350e+09,6.669167e+09,6.875338e+08,7.237071e+08,5.976350e+09,6.669167e+09,1.817068e+09,1.928032e+09,2.131279e+10,2.483020e+10


Now the time series are ready to be used as an input for the vulnerability assessment in the next notebook 'DemoVulnerabilityAssessments.ipynb'. The data frame also contains quantile time series.We can save its as an output as .csv

In [33]:
med_mod_data.to_csv('~/projects/NC_Submission/Climada_papers/Test/DEMO_ModelMedianSubregions.csv')